In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.fundaments.brace import Brace as _Brace
from everest.ptolemaic.sig import Sig as _Sig

In [ ]:
class ClassSlyce

In [ ]:
from everest.incision import ChainIncisable as _ChainIncisable
from everest.utilities import caching as _caching, Slc as _Slc

from everest.ptolemaic.fundaments.brace import Brace as _Brace
from everest.ptolemaic.fundaments.thing import Thing as _Thing
from everest.ptolemaic.fundaments.floatt import Floatt as _Floatt

from everest.ptolemaic.sprite import Sprite as _Sprite

In [ ]:
class IndexSpace(_ChainIncisable, metaclass=_Sprite):

    space: _Brace[::_Thing]
    index: _Floatt.Closed[0.:]

    @property
    @_caching.soft_cache()
    def __incision_manager__(self, /):
        return _Brace[dict(
            space=_idx[::self.space],
            index=_idx[::self.index],
            )]

In [ ]:
myobj = IndexSpace(
    _Brace[dict(x=_idx[::_Floatt], y=_idx[::_Floatt])],
    _Floatt[0.:]
    )
myobj

In [ ]:
myobj[(1., 2.), 3.:]

In [ ]:
_Floatt[0.:]

In [ ]:
_Floatt.Closed[0.:]